In [1]:
from synsense.net.auto import AutoSNN
from synsense.net.dataset import Dataset_Texture_Stream
import torch.nn as nn
import torch
print(torch.cuda.is_available())

True


/home/xingchen/anaconda3/envs/synsense/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Split train and test data into lists

In [2]:
from user import split_train_test
root = r'/media/xingchen/T7 Shield/Neuromorphic Data/Xingchen/processed/0220_dataset_iros/general_sliderotate'
train_size = 0.7
random_seed = 42
train_list, test_list = split_train_test(root, train_size, random_seed)

Dataset creation

In [3]:
device = "pc"
# gridsize = (20, 20)
gridsize = (130, 130)
after_crop_size = (260, 260, 1)
# n_time_bins = 40
n_time_bins = 100

train_dataset = Dataset_Texture_Stream(train_list, device, gridsize, after_crop_size, n_time_bins)
test_dataset = Dataset_Texture_Stream(test_list, device, gridsize, after_crop_size, n_time_bins)

print(len(train_dataset), len(test_dataset))
print(train_dataset[0][0].shape, train_dataset[0][1], 
      train_dataset[0][0].max(), train_dataset[0][0].sum())

3500 1500
torch.Size([100, 1, 130, 130]) tensor(2) tensor(4.) tensor(457.)


In [21]:
batch_size = 600
auto = AutoSNN(batch_size)
model_path = r'./models/0127_24000_event_stream_us.pt'
device = "cuda:0"
k_folds = 3
epochs = 9
step_size = 10
cnn = nn.Sequential(
    nn.Conv2d(1, 8, 3, 1, bias=False),
    nn.ReLU(),
    nn.AvgPool2d(2, 2),
    nn.Conv2d(8, 16, 3, 1, bias=False),
    nn.ReLU(),
    nn.AvgPool2d(2, 2),
    nn.Flatten(),
    nn.Linear(144, 256, bias=False),
    nn.ReLU(),
    nn.Linear(256, 72, bias=False),
    nn.ReLU(),
    nn.Linear(72, 10, bias=False)
)

train_accs, val_accs = auto.train(
    model_path=model_path, 
    model=cnn, 
    train_dataset=train_dataset, 
    device=device, 
    k_folds=k_folds, 
    epochs=epochs,
    step_size = step_size
    )

K fold: 0
epoch: 0; Accuracy:  17.52%; Loss:  39.43; current_lr:  0.001000
epoch: 1; Accuracy:  29.37%; Loss:  34.87; current_lr:  0.001000
epoch: 2; Accuracy:  37.59%; Loss:  31.07; current_lr:  0.001000
epoch: 3; Accuracy:  49.57%; Loss:  25.41; current_lr:  0.001000
epoch: 4; Accuracy:  57.86%; Loss:  20.79; current_lr:  0.001000
epoch: 5; Accuracy:  64.77%; Loss:  17.73; current_lr:  0.001000
epoch: 6; Accuracy:  69.05%; Loss:  15.72; current_lr:  0.001000
epoch: 7; Accuracy:  69.46%; Loss:  15.70; current_lr:  0.001000
epoch: 8; Accuracy:  73.39%; Loss:  13.22; current_lr:  0.001000
val accuray:  75.74%
K fold: 1
epoch: 0; Accuracy:  16.70%; Loss:  39.52; current_lr:  0.001000
epoch: 1; Accuracy:  29.89%; Loss:  34.82; current_lr:  0.001000
epoch: 2; Accuracy:  38.53%; Loss:  30.85; current_lr:  0.001000
epoch: 3; Accuracy:  47.86%; Loss:  26.49; current_lr:  0.001000
epoch: 4; Accuracy:  55.93%; Loss:  21.79; current_lr:  0.001000
epoch: 5; Accuracy:  63.72%; Loss:  18.14; curren

In [26]:
train_accs, val_accs

([tensor(0.7339, device='cuda:0'),
  tensor(0.7282, device='cuda:0'),
  tensor(0.7088, device='cuda:0')],
 [tensor(0.7574, device='cuda:0'),
  tensor(0.7357, device='cuda:0'),
  tensor(0.7344, device='cuda:0')])

Test on PC

In [31]:
model_path = r'./models/0127_24000_event_stream_us.pt'
device = "cuda:0"
auto.test(
    test_dataset,
    model_path,
    device
)

test accuracy:  74.61%


In [4]:
auto = AutoSNN(100)
model_path = r'/home/xingchen/PhD/synsense/test/0117-xytp/models/general_slide-gs130-tl100-ddp.pt'
device = "cuda:1"
auto.test(
    test_dataset,
    model_path,
    device
)

/home/xingchen/PhD/synsense/synsense/net/auto.py:249: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  snn = torch.load(model_path).to(device)


test accuracy:  79.67%


Test on board

In [ ]:
from synsense.board.deploy import DeploySpeck
from torch.utils.data.dataset import Subset

model_path = r'/home/xingchen/PhD/synsense/test/0117-xytp/models/general_sliderotate-gs130-tl100-ddp.pt'
platform = "speck"
test_dataset = Dataset_Texture_Stream(test_list, platform, gridsize, after_crop_size, n_time_bins)
# sub_test_dataset = Subset(test_dataset, range(0, 7200, 25))
sub_test_dataset = Subset(test_dataset, range(0, 1500, 5))

# ds = DeploySpeck((1,20,20))
ds = DeploySpeck((1,130,130))
board = ds.open_device()
dynapcnn = ds.deploy_net_on_Speck(model_path)
ps = ds.infer(sub_test_dataset, dynapcnn, board)

In [33]:
ds.close_device(board)

Board closed successfully.
